In [2]:
import numpy as np
np.array([[1,2,3]]).shape

(1, 3)

In [4]:
import numpy as np
a = np.ones((10,3))
b = np.ones((10))
a + b[:,None]

array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]])

In [2]:
for i in range(100):
    for j in range(100):
        for k in range(100):
            a = 1

In [6]:
np.sum(np.ones((10,3)), axis=0)

array([10., 10., 10.])

In [11]:
np.zeros(5)[:]

array([0., 0., 0., 0., 0.])